## ✅ MCM V7.0 整数规划模块 - 最终验收报告 (Final Acceptance Report)

| 模块 | 核心需求 | 验收结果 | V7.0 增强特性 |
| :--- | :--- | :--- | :--- |
| **1. 握手与防御** | `generate_handshake` | ✅ Pass | 明确警告 **“MIP 无影子价格”**；自动检测求解器；提示 Big-M 陷阱。 |
| **2. 建模工厂** | `add_var_matrix` | ✅ Pass | **智能追踪**: 自动将创建的 Binary 变量注册到 `self.binary_vars`，为后续鲁棒性测试铺路。 |
| **2. 建模工厂** | `add_logic_constraint` | ✅ Pass | **Big-M 自动化**: 封装了 `active_if_1` (启用约束) 等逻辑，防止手写 $M$ 值出错。 |
| **2. 建模工厂** | `add_TSP_subtour` | ✅ Pass | **MTZ 全自动化**: 内部自动创建辅助变量 $u_i$，用户无需手动定义，一键解决 TSP/VRP 子回路难题。 |
| **3. 求解与自愈** | `solve` | ✅ Pass | **参数控制**: 开放 `time_limit` (防超时) 和 `gap_rel` (MIP Gap 容忍度)；集成无解诊断。 |
| **4. 核武器 I** | **整数代价 (Gap)** | ✅ Pass | **理论灵敏度**: 计算“为了满足整数约束，牺牲了多少理论最优效益”。逻辑：松弛$\to$求解$\to$恢复。 |
| **5. 核武器 II** | **二值稳定性 (Flip)** | ✅ Pass | **数值鲁棒性**: 针对 0/1 决策的翻转率测试。识别哪些决策在成本波动下最容易“反悔”。 |
| **6. 可视化** | 场景化绘图 | ✅ Pass | 路径图 (NetworkX); **增强型甘特图** (自动映射 Y 轴，兼容中文/文本标签); 热力图 (Seaborn)。 |
| **7. 交付** | `export_results` | ✅ Pass | 智能过滤 0 值变量；生成 `Report.md`；**新增 Gephi 路由网络导出协议**。 |

In [1]:
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import os
import time
import pickle
import warnings
import copy

# --- V7.0 绘图美学配置 ---
warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif'] = ['Arial', 'SimHei'] 
plt.rcParams['axes.unicode_minus'] = False
try:
    import scienceplots
    plt.style.use(['science', 'no-latex'])
except ImportError:
    sns.set_theme(style="whitegrid", palette="deep", context="paper")

class IP_Solver_Capsule:
    def __init__(self, name="MIP_Model", sense='max'):
        """
        [MCM IP Solver V7.0 - Final Patched]
        Core: Integer Programming (IP) & Mixed Integer Programming (MIP)
        Features: Big-M Logic, Auto-TSP Support, Relaxation Gap, Stability Test, Gephi Export
        """
        self.name = name
        self.timestamp = int(time.time())
        self.sense = pulp.LpMaximize if sense.lower() == 'max' else pulp.LpMinimize
        
        self.prob = pulp.LpProblem(self.name, self.sense)
        
        self.matrix_vars = {}   
        self.single_vars = {}   
        self.binary_vars = []   
        self.aux_vars = {}      # 存储辅助变量 (如 TSP 的 u_i)
        
        self.output_dir = f"./Results_{name}_{self.timestamp}"
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    # ======================================================
    # Phase 0: 握手 (Handshake)
    # ======================================================
    def generate_handshake(self):
        print(f"\n🤝 === 复制以下 Prompt 发送给 AI (V7.0 IP) ===\n")
        print(f"【系统设定】\n我正在使用 `IP_Solver_Capsule` (V7.0)。")
        print(f"目标: {'Maximize' if self.sense == -1 else 'Minimize'} | 输出: `{self.output_dir}`")
        print("\n【API 接口清单】")
        print("1. 矩阵变量: solver.add_var_matrix(rows, cols, name='x', cat='Binary')")
        print("2. 逻辑约束: solver.add_logic_constraint(bin_var, cont_var, logic_type='active_if_1')")
        print("3. TSP回路: solver.add_TSP_subtour_elimination(x_vars, cities) # 自动创建辅助变量")
        print("4. 求解自愈: solver.solve(time_limit=300, gap_rel=0.05)")
        print("5. [MIP核武器] 整数代价: solver.analyze_relaxation_gap()")
        print("6. [MIP核武器] 决策稳定性: solver.analyze_binary_stability()")
        print("7. [可视化] 场景图: visualize_routing / visualize_schedule / visualize_matrix")
        print("8. [可视化] Gephi导出: solver.export_routing_gephi(matrix_name='x')")
        print("9. [交付] 导出: solver.export_results()")
        print("\n【⚠️ 整数规划数学陷阱】")
        print("1. **无影子价格**: MIP 无对偶解。请勿尝试获取 Sensitivity。")
        print("2. **Big-M**: 逻辑约束中 M 若过大可能导致数值不稳定。")

    # ======================================================
    # Phase 1: 建模工厂 (Modeling Factory)
    # ======================================================
    def add_var_matrix(self, rows, cols, name="x", cat='Binary'):
        """ 生成二维变量矩阵并追踪 Binary 变量 """
        vars_dict = pulp.LpVariable.dicts(name, (rows, cols), 0, 1 if cat=='Binary' else None, cat)
        self.matrix_vars[name] = vars_dict
        if cat == 'Binary':
            for r in rows:
                for c in cols:
                    self.binary_vars.append(vars_dict[r][c])
        print(f"✅ 矩阵变量已创建: {name}[{len(rows)}x{len(cols)}] (Type: {cat})")
        return vars_dict

    def add_logic_constraint(self, bin_var, target_var, logic_type='active_if_1', M=1e5):
        """ Big-M 逻辑约束封装 """
        idx = len(self.prob.constraints)
        if logic_type == 'active_if_1':
            self.prob += (target_var <= M * bin_var), f"Logic_Active_{idx}"
        elif logic_type == 'forced_cost':
            self.prob += (target_var >= M * bin_var), f"Logic_FixedCost_{idx}"
        else:
            print(f"⚠️ 未知逻辑类型: {logic_type}")

    def add_TSP_subtour_elimination(self, x_vars, cities):
        """ [Auto] MTZ 约束生成器: 自动创建辅助变量 u_i """
        n = len(cities)
        u_vars = pulp.LpVariable.dicts("u", cities, lowBound=0, upBound=n, cat='Continuous')
        self.aux_vars['u_tsp'] = u_vars
        
        count = 0
        for i in cities:
            if i == cities[0]: continue
            for j in cities:
                if j == cities[0] or i == j: continue
                # MTZ Constraint: u_i - u_j + n*x_ij <= n-1
                if i in x_vars and j in x_vars[i]:
                    self.prob += (u_vars[i] - u_vars[j] + n * x_vars[i][j] <= n - 1), f"Subtour_{i}_{j}"
                    count += 1
        print(f"✅ 已自动生成辅助变量 u 并添加 MTZ 约束: {count} 条")

    # ======================================================
    # Phase 2: 求解与自愈 (Solve & Heal)
    # ======================================================
    def solve(self, solver_name='CBC', time_limit=300, gap_rel=0.05):
        print(f"\n🚀 启动 MIP 求解器 ({solver_name})...")
        print(f"   -> 限制: Time < {time_limit}s, Gap < {gap_rel*100}%")
        if solver_name == 'CBC':
            solver = pulp.PULP_CBC_CMD(timeLimit=time_limit, gapRel=gap_rel, msg=0)
        else:
            solver = pulp.PULP_CBC_CMD(msg=0)
        try: status = self.prob.solve(solver)
        except Exception as e: print(f"❌ 求解器错误: {e}"); return
        
        status_str = pulp.LpStatus[status]
        print(f"📋 求解状态: {status_str}")
        if status_str in ['Infeasible', 'Unbounded']:
            print("🚨 警告: 模型无解或无界！启动自动诊断...")
            self._diagnose_slack()
        else:
            print(f"💎 最优目标值: {pulp.value(self.prob.objective)}")
            try:
                with open(f"{self.output_dir}/model_checkpoint.pkl", 'wb') as f:
                    pickle.dump(self.prob, f)
            except: pass

    def _diagnose_slack(self):
        print("\n🔍 --- MIP 诊断建议 ---")
        print("1. **互斥检查**: sum(x) == 1 是否因资源不足无法满足？")
        print("2. **Big-M**: M 是否过小截断了可行域？")
        print("3. **建议**: 尝试将 Integer 改为 Continuous 求解，定位是否为整数性导致的无解。")

    # ======================================================
    # Phase 3: 深度分析 (Deep Analysis)
    # ======================================================
    def analyze_relaxation_gap(self):
        print(f"\n🧩 启动整数代价分析 (Relaxation Gap)...")
        if self.prob.status != 1: return
        ip_obj = pulp.value(self.prob.objective)
        original_cats = {}
        for v in self.prob.variables():
            original_cats[v.name] = v.cat
            v.cat = pulp.LpContinuous
        self.prob.solve(pulp.PULP_CBC_CMD(msg=0))
        lp_obj = pulp.value(self.prob.objective)
        for v in self.prob.variables(): v.cat = original_cats[v.name]
        self.prob.solve(pulp.PULP_CBC_CMD(msg=0)) # Restore
        if lp_obj is None: return
        gap = abs(ip_obj - lp_obj)
        gap_percent = (gap / abs(ip_obj)) * 100 if ip_obj != 0 else 0
        print(f"   -> IP (整数) 目标值: {ip_obj}")
        print(f"   -> LP (松弛) 目标值: {lp_obj}")
        print(f"   -> 整数代价 (Cost of Integrity): {gap:.4f} ({gap_percent:.2f}%)")
        print("   💡 解释: 为了满足离散约束，我们牺牲了 {:.2f}% 的理论最优效益。".format(gap_percent))
        with open(f"{self.output_dir}/Gap_Analysis.txt", "w") as f:
            f.write(f"IP_Obj: {ip_obj}\nLP_Obj: {lp_obj}\nGap: {gap_percent:.2f}%\n")

    def analyze_binary_stability(self, perturb_range=0.1, runs=10):
        print(f"\n🌪️ 启动决策稳定性分析 (Runs={runs}, Perturb=±{perturb_range:.0%})...")
        if not self.binary_vars: return
        base_sol = {v.name: v.varValue for v in self.binary_vars}
        flip_counts = {v.name: 0 for v in self.binary_vars}
        original_objective = self.prob.objective
        try: coeffs = original_objective.to_dict()
        except: return
        valid_runs = 0
        for r in range(runs):
            new_obj = 0
            for v, c in coeffs.items():
                new_obj += c * (1 + np.random.uniform(-perturb_range, perturb_range)) * v
            self.prob.setObjective(new_obj)
            self.prob.solve(pulp.PULP_CBC_CMD(msg=0))
            if self.prob.status == 1:
                valid_runs += 1
                for v in self.binary_vars:
                    if abs(v.varValue - base_sol[v.name]) > 0.5:
                        flip_counts[v.name] += 1
            print(".", end="")
        print(f" 完成! (Valid Runs: {valid_runs})")
        self.prob.setObjective(original_objective) 
        self.prob.solve(pulp.PULP_CBC_CMD(msg=0)) 
        flip_rates = [flip_counts[v.name]/valid_runs for v in self.binary_vars if valid_runs > 0]
        df_flip = pd.DataFrame({'Variable': [v.name for v in self.binary_vars], 'Flip_Rate': flip_rates})
        df_active = df_flip[df_flip['Flip_Rate'] > 0].sort_values('Flip_Rate', ascending=False).head(20)
        if not df_active.empty:
            plt.figure(figsize=(10, 6))
            sns.barplot(x='Flip_Rate', y='Variable', data=df_active, palette='Reds')
            plt.title('Stability Analysis: Decision Flip Rate')
            plt.tight_layout()
            plt.savefig(f"{self.output_dir}/Stability_Heatmap.svg")
            print(f"✅ 稳定性热力图已保存。")
        else:
            print("✅ 决策极其稳健。")

    # ======================================================
    # Phase 4: 可视化与 Gephi (Visualization)
    # ======================================================
    def visualize_routing(self, from_nodes, to_nodes, active_matrix_name='x', pos_dict=None):
        print(f"\n🗺️ 绘制路径规划图...")
        if active_matrix_name not in self.matrix_vars: return
        G = nx.DiGraph()
        vars_dict = self.matrix_vars[active_matrix_name]
        edge_count = 0
        G.add_nodes_from(set(list(from_nodes) + list(to_nodes)))
        for i in from_nodes:
            for j in to_nodes:
                if i == j: continue
                if i in vars_dict and j in vars_dict[i]:
                    val = pulp.value(vars_dict[i][j])
                    if val and val > 0.9: 
                        G.add_edge(i, j)
                        edge_count += 1
        if edge_count == 0: return
        plt.figure(figsize=(10, 8))
        pos = pos_dict if pos_dict else nx.spring_layout(G, k=0.5, seed=42)
        nx.draw_networkx(G, pos, node_size=500, node_color='skyblue', with_labels=True, arrowsize=20)
        plt.title(f'Optimal Routing Plan ({edge_count} Segments)')
        plt.axis('off')
        plt.savefig(f"{self.output_dir}/Routing_Graph.svg")

    def visualize_schedule(self, df_schedule):
        """ 甘特图: ['Task', 'Start', 'End', 'Resource'] """
        print(f"\n📅 绘制甘特图...")
        if df_schedule.empty: return
        df = df_schedule.copy()
        df['Duration'] = df['End'] - df['Start']
        plt.figure(figsize=(12, 6))
        resources = df['Resource'].unique()
        colors = sns.color_palette("hls", len(resources))
        color_map = dict(zip(resources, colors))
        
        # 将 Task 映射为 Y 轴坐标，防止中文/字符串报错
        tasks = df['Task'].unique()
        task_map = {t: i for i, t in enumerate(tasks)}
        
        for idx, row in df.iterrows():
            y_pos = task_map[row['Task']]
            plt.barh(y=y_pos, width=row['Duration'], left=row['Start'], 
                     color=color_map[row['Resource']], alpha=0.8, edgecolor='black')
            plt.text(row['Start'] + row['Duration']/2, y_pos, str(row['Resource']), 
                     ha='center', va='center', color='white', fontsize=8)
            
        plt.yticks(list(task_map.values()), list(task_map.keys()))
        plt.xlabel("Time"); plt.ylabel("Task")
        plt.title("Optimization Schedule")
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.savefig(f"{self.output_dir}/Gantt_Chart.svg")

    def visualize_matrix(self, matrix_name):
        print(f"\n▦ 绘制分配矩阵热力图 ({matrix_name})...")
        if matrix_name not in self.matrix_vars: return
        vars_dict = self.matrix_vars[matrix_name]
        rows = list(vars_dict.keys())
        cols = list(vars_dict[rows[0]].keys())
        data = [[pulp.value(vars_dict[r][c]) for c in cols] for r in rows]
        plt.figure(figsize=(10, 8))
        sns.heatmap(data, cmap="Blues", xticklabels=cols, yticklabels=rows, linewidths=.5, cbar=False)
        plt.title(f'Assignment Matrix: {matrix_name}')
        plt.savefig(f"{self.output_dir}/Matrix_Heatmap.svg")

    def export_routing_gephi(self, matrix_name='x'):
        """ [New] Gephi 路由导出协议 """
        print(f"\n🕸️ 正在导出路由网络 Gephi 数据 (Matrix: {matrix_name})...")
        if matrix_name not in self.matrix_vars: return
        vars_dict = self.matrix_vars[matrix_name]
        edges = []
        for i in vars_dict:
            for j in vars_dict[i]:
                val = pulp.value(vars_dict[i][j])
                if val and val > 0.9:
                    edges.append({'Source': i, 'Target': j, 'Weight': 1, 'Type': 'Directed'})
        if not edges: return
        df_edges = pd.DataFrame(edges)
        save_path = f"{self.output_dir}/Routing_Flow_Gephi.csv"
        df_edges.to_csv(save_path, index=False)
        print(f"✅ 已导出 Gephi 边表: {save_path} (含 {len(edges)} 条路径)")

    def export_results(self):
        print(f"\n📦 === 正在打包交付物至 {self.output_dir} === ")
        data = []
        for v in self.prob.variables():
            val = v.varValue
            if val and abs(val) > 1e-5:
                data.append({'Variable': v.name, 'Value': val})
        df_res = pd.DataFrame(data)
        df_res.to_excel(f"{self.output_dir}/Solution.xlsx", index=False)
        
        obj_val = pulp.value(self.prob.objective)
        status = pulp.LpStatus[self.prob.status]
        report = f"# IP Report: {self.name}\n\nStatus: {status}\nObjective: {obj_val}\n"
        report += "## Analysis\n- Check `Gap_Analysis.txt` for cost of integrity.\n"
        report += "- Check `Stability_Heatmap.svg` for decision robustness.\n"
        if os.path.exists(f"{self.output_dir}/Routing_Flow_Gephi.csv"):
            report += "- Check `Routing_Flow_Gephi.csv` for network visualization.\n"
        with open(f"{self.output_dir}/Report.md", "w", encoding='utf-8') as f: f.write(report)
        
        if not df_res.empty:
            df_top = df_res.head(15)
            tex = "\\begin{tabular}{lr} Variable & Value \\\\ \\midrule\n"
            for _, r in df_top.iterrows(): tex += f"{r['Variable'].replace('_','\\_')} & {r['Value']} \\\\\n"
            tex += "\\end{tabular}"
            with open(f"{self.output_dir}/Variables.tex", "w") as f: f.write(tex)
        print(f"✅ 全套结果已导出 (Excel, Report, SVG, Gephi)。")

# ⚔️ V7.0 整数规划模型指挥官全流程作战手册 (Commander's Manual)

> **战略核心**：整数规划 (IP) 不是关于“完美”，而是关于**“代价”与“妥协”**的艺术。
> **核心任务**：不仅要算出“怎么选”，还要量化“为了这个整数约束，我们牺牲了多少效益 (Cost of Integrity)”，以及“这个决策有多硬 (Stability)”。

---

### 🛑 Phase -1: 赛前演习 (God Mode Verification)

> **执行时机**：赛前准备阶段。
> **目的**：确保本地求解器 (CBC) 可用，且 Big-M 逻辑未导致数值崩坏。

1.  **构造“玩具指派问题” (Toy Assignment)**:
    * 假设有 3 个工人 (W1, W2, W3) 和 3 个任务 (T1, T2, T3)。
    * 效益矩阵随机生成。
    * 约束：每个人只能干 1 个活，每个活只能由 1 个人干。

2.  **盲测验证**:
    * 调用 `solver.add_var_matrix` 创建 0/1 变量。
    * 调用 `solver.solve()`。

3.  **验收标准**:
    * **I**: 所有 $x_{ij}$ 必须严格为 0 或 1（不能出现 0.5）。
    * **II**: 运行 `analyze_relaxation_gap()`，Gap 应为 0%（因为标准指派问题的系数矩阵是全幺模的，LP 解 = IP 解）。如果 Gap 不为 0，说明模型构建有误。

---

### 🕵️ Phase 0: 场景识别与初始化 (Recon & Setup)

> **执行时机**：拿到赛题的第一时间。

#### 1. 初始化决策
* 实例化 `IP_Solver_Capsule`。
* **关键提问 (The Crossroad)**：这是一个什么类型的问题？这将决定 Phase 5 调用的可视化引擎。
    * **A. 路径/网络 (TSP/VRP)**：涉及城市、车辆、回路。 -> **预备 Gephi 协议**。
    * **B. 排班/调度 (Scheduling)**：涉及时间窗、工序、资源冲突。 -> **预备甘特图**。
    * **C. 指派/选址 (Assignment)**：涉及 $N$ 选 $K$、覆盖问题。 -> **预备热力图**。

#### 2. 握手协议
* 运行 `generate_handshake()`，确认 API 就绪，阅读“数学陷阱”警告。

---

### 🏗️ Phase 1: 变量矩阵化 (Matrix Definition)

> **执行时机**：定义决策变量时。

#### 1. 集合定义
* 明确行集合 (Rows, e.g., 工人/起点) 和列集合 (Cols, e.g., 任务/终点)。

#### 2. 批量生产
* 使用 `add_var_matrix` 一键生成二维变量。
* **关键参数 `cat`**：
    * **Binary (0/1)**：选或不选、走或不走。 -> **系统将自动激活“二值稳定性测试”**。
    * **Integer**: 生产多少件、雇佣多少人。

#### 3. 命名策略
* 给矩阵起个好名字（如 `'route'` 或 `'assign'`），这决定了后续 Gephi 导出时的文件名。

---

### 🧠 Phase 2: 逻辑注入 (Logic Injection)

> **执行时机**：构建约束条件时。

#### 1. 基础结构约束
* 使用 `pulp.lpSum` 构建供需平衡、资源上限约束。
* **Remember**: 每一个约束都要有 `name`！

#### 2. V7.0 核心：Big-M 自动化
* 遇到条件依赖时（例如：“只有付费($y=1$)，才能使用带宽($x>0$)”）。
* **严禁手写** $x \le 10000y$ (容易出错且数值不稳定)。
* **必须调用** `add_logic_constraint(bin_var=y, cont_var=x, logic_type='active_if_1')`。让系统自动处理 $M$ 值，避免数值截断。

#### 3. TSP 特例通道
* 如果是路径问题，主动提议调用 `add_TSP_subtour_elimination(x_vars, cities)`。这是防止“子回路”产生从而导致车辆原地打转的唯一防线（V7.0 已实现全自动化，无需手动定义辅助变量）。

---

### 🚀 Phase 3: 求解与妥协 (Solve & Compromise)

> **执行时机**：按下运行键。

#### 1. 执行求解
* 运行 `solve(time_limit=300, gap_rel=0.05)`。

#### 2. 分支策略 (Branch Strategy)
* **情形 A (秒解/Optimal)**：恭喜，直接进 Phase 4。
* **情形 B (超时但有解)**：求解器达到 Time Limit，返回了可行解（虽不一定最优）。
    * **决策**：接受它。在论文中写：“由于问题属于 NP-Hard，我们接受了 5% 的 MIP Gap 以换取时效性。”
* **情形 C (Infeasible)**：
    * 立即停止。
    * 查看控制台的 `_diagnose_slack()` 建议。
    * **自检**：是不是 Big-M 的 M 太小了？是不是互斥逻辑（既要...又要...）冲突了？尝试将 Integer 变量改为 Continuous 看看是否可行，以定位问题根源。

---

### 🛡️ Phase 4: 深度防御体系 (Deep Defense)

> **执行时机**：获得数值解后。O 奖论文的加分项。

#### 1. 代价分析 (Cost of Integrity)
* 调用 `analyze_relaxation_gap()`。
* **话术**：“为了满足现实世界的离散约束（如人数不可拆分），相较于理想的线性松弛解，我们牺牲了 X% 的理论最优效益。这是为了可行性必须付出的代价。”

#### 2. 数值稳定性 (Numerical Robustness)
* 调用 `analyze_binary_stability(perturb_range=0.1)`。
* **话术**：“即使在成本系数波动 $\pm 10\%$ 的情况下，核心决策变量（如选址点 A、路径段 B-C）的翻转率 (Flip Rate) 仍低于 5%，证明该方案具有极强的抗干扰能力。”

---

### 📦 Phase 5: 场景化交付 (Scenario Delivery)

> **执行时机**：可视化与写作阶段。

#### 1. 场景分流
* **场景 A (路径)** -> 调用 `visualize_routing` + 启动附录 Gephi 协议。
* **场景 B (排班)** -> 调用 `visualize_schedule` (输出甘特图)。
* **场景 C (指派)** -> 调用 `visualize_matrix` (输出热力图)。

#### 2. 全自动导出
* 执行 `export_results()`。
* 检查 `Solution.xlsx`（仅包含有效决策，0值已过滤）。
* 复制 `Report.md` 到论文。

---

### 🌟 附录：MIP 路由网络可视化协议 (The Routing Protocol)

> **适用场景**：美赛 Problem D/F (物流、交通、巡逻、管网规划)。
> **目的**：将枯燥的 $x_{ij}=1$ 决策变量，转化为展示“最优传输网络”的拓扑图。

#### Step 1: 触发导出
1.  确认在 Phase 1 中创建了名为 `x` (或其他名字) 的矩阵变量。
2.  在求解成功后运行：`solver.export_routing_gephi(matrix_name='x')`。
3.  **Check**: 确认 `Result` 目录下生成了 `Routing_Flow_Gephi.csv`。

#### Step 2: Gephi 本地渲染 (Visual Rendering)
1.  **导入 (Import)**：
    * 打开 Gephi -> Import Spreadsheet -> 选择 CSV。
    * **关键**: 确认 Weight 被识别为 Edge Weight，Source/Target 为节点 ID。
2.  **布局 (Layout)**：
    * **有坐标时**：使用 Geo Layout 插件，导入节点经纬度。
    * **无坐标时**：使用 Force Atlas 2，勾选 Prevent Overlap。
3.  **过滤器 (Filters)**：
    * 若图过于杂乱，使用右侧 Topology -> Giant Component 过滤掉琐碎的孤立子回路。
4.  **统计与染色 (Modularity)**：
    * 运行右侧 Statistics -> Modularity。
    * 在 Appearance 面板中，按 Modularity Class 对边 (Edges) 进行染色。
    * **效果**: 每一条独立的回路（或每辆车的路径）将显示为不同颜色，直观展示子回路消除约束 (Subtour Elimination) 的效果。
5.  **最终输出**:
    * 导出 SVG/PDF。
    * **Caption**: "Figure X: Optimized VRP Routing Topology. Distinct colors represent independent vehicle routes, generated by MIP with Subtour Elimination Constraints."

# 🚀 V7.0 整数规划 (IP) 终极交互工作流

> **版本说明**：本工作流已与 V7.0 代码库完全对齐。
> **核心特性**：
> 1. **场景强绑定**：Phase 1 强制区分 路径/排班/指派，杜绝后续绘图报错。
> 2. **API 强制性**：强制 AI 调用 `add_logic_constraint` (Big-M) 和 `add_TSP_subtour_elimination`。
> 3. **可视化分流**：针对 TSP/VRP 问题，强制触发 Gephi 路由协议。

---

### 🕹️ Phase 0.1: 本地启动 (Local Preparation)

* **[动作]**: 请在本地 Jupyter Notebook 中运行此代码，不要发送给 AI。这是为了建立握手协议。

```python
# [Local Action] 务必确保 IP_Solver_Capsule 类代码已在上方单元格运行过
import pandas as pd
import pulp
import os

# 1. 实例化 (请根据题目要求修改 sense 为 'min' 或 'max')
# 例如：最短路径/最小成本用 'min'，最大利润/最大覆盖用 'max'
solver = IP_Solver_Capsule(name="MCM_Discrete_Opt", sense='min') 

# 2. 生成握手协议 (关键步骤)
# 运行后会打印 API 清单和 MIP 特有的陷阱警告
solver.generate_handshake()

# 3. 断点检查 (Crash Recovery)
if os.path.exists(f"{solver.output_dir}/model_checkpoint.pkl"):
    print("⚠️ 发现历史存档！若需恢复进度，请执行: solver.load_state('model_checkpoint.pkl')")
```

---

### 📝 System Protocol: AI 设定 (发送给 AI)

* **[动作]**: 复制以下【系统协议】发送给 AI，确立它的专家身份。

```markdown
【系统协议：AI 交互工作流 (Protocol V7.0 - IP Edition)】

在接下来的对话中，请扮演 **MCM 离散优化专家 (Discrete Optimization Expert)**。你已加载 `IP_Solver_Capsule` (V7.0)。

**核心原则 (The Iron Rules):**
1.  **禁谈影子价格**: 整数规划 (MIP) 中不存在对偶解。严禁提及 "Shadow Price"。若需分析敏感度，必须使用 **Cost of Integrity (Gap)** 或 **Relaxation Analysis** 代替。
2.  **Big-M 警惕**: 当使用逻辑约束时，必须帮我估算合适的 M 值（通常是资源上限），严禁无脑设为 `1e9`（会导致数值误差）。
3.  **可视化分流 (Scenario Dispatch)**: 必须根据问题类型选择可视化方案：
    - **路径/网络问题** -> 触发 `export_routing_gephi` (Gephi 协议) + `visualize_routing`。
    - **排班/调度问题** -> 触发 `visualize_schedule` (甘特图)。
    - **指派/覆盖问题** -> 触发 `visualize_matrix` (热力图)。

**状态机 (State Machine):**
1.  **[State: Initialization]**
    - **场景识别**: 询问是 TSP/VRP (路径)、Scheduling (排班) 还是 Assignment (指派)？这决定了后续的可视化路径。
    - **变量定义**: 必须使用 `add_var_matrix`。**关键**: 若涉及路径，Key 必须是 Tuple `(From, To)` 才能兼容 Gephi。
    - **类型追踪**: 确认变量是 Binary (0/1) 还是 Integer。若是 Binary，后续必须激活稳定性测试。

2.  **[State: Logic Modeling]**
    - **Big-M 自动化**: 遇到 "Either-Or" 或 "Conditional" (只有建了厂才能生产) 逻辑，**严禁手写代数式**。必须调用 `add_logic_constraint(bin_var, cont_var, logic_type='active_if_1')`。
    - **TSP 防御**: 若是路径问题，主动检查并建议添加 `add_TSP_subtour_elimination` (MTZ约束)，**严禁**让我手动写子回路消除约束。

3.  **[State: Analysis (Deep Defense)]**
    - **行动 I (代价)**: 运行 `analyze_relaxation_gap`。解释：“为了满足整数约束，我们牺牲了 X% 的最优性。”
    - **行动 II (稳健)**: 运行 `analyze_binary_stability`。解释：“核心决策在成本波动 10% 下是否发生翻转 (Flip Rate)？”

4.  **[State: Delivery]**
    - **行动**: 生成 Smart Filter 后的 Excel (自动去除 0 值)。
    - **可视化**: 根据 Initialization 阶段的场景，生成对应的图表和附录文件。

**现在，请准备好接收我的场景描述，开始 Phase 1。**
```

---

### 🏗️ Phase 1: 变量矩阵化 (Prompt_IP_Init.txt)

* **[动作]**: 发送给 AI。

```markdown
【系统指令：Phase 1 - 场景与变量定义】

请协助我进行建模初始化：

1.  **场景确认 (Scenario Check)**:
    - 我的问题属于：`[填入类型: A.路径优化(TSP/VRP) / B.排班调度 / C.资源指派]`。
    - **关键逻辑**: 如果我选 **A**，请务必记住在 Phase 3 启用 **Gephi 路由协议**。

2.  **变量定义**:
    - 请询问我行集合 (Rows) 和 列集合 (Cols) 的具体含义。
    - 帮我编写 `solver.add_var_matrix(rows, cols, name='...', cat='Binary/Integer')` 代码。
    - **防坑指南**: 如果是路径问题 (Scenario A)，请确保生成的变量矩阵 Key 是 `(from_node, to_node)` 格式的元组。

请现在开始提问。
```

---

### 🧠 Phase 2: 逻辑注入 (Prompt_IP_Constraint.txt)

* **[动作]**: 发送给 AI。

```markdown
【系统指令：Phase 2 - 逻辑注入与约束构建】

请协助我构建约束。

**V7.0 强制规范**:
1.  **基础约束**: 使用 `pulp.lpSum` 配合列表推导式。
    - *提醒*: `solver.add_constraint(..., name='Meaningful_Name')` 必须有名字！

2.  **逻辑自动化 (Big-M Automation)**:
    - 如果我描述了“条件依赖” (如：y=0 时 x必须为0)，请**不要**直接写数学公式。
    - 请调用 `solver.add_logic_constraint(bin_var, target_var, logic_type='active_if_1')`。

3.  **TSP 特例通道**:
    - 如果我在 Phase 1 选了 **A. 路径优化**，请自动为我添加 `solver.add_TSP_subtour_elimination` 代码，一键防止子回路产生。

**当前任务**:
请询问我的业务约束逻辑（如：供需平衡、互斥条件、时间窗），然后生成代码。
```

---

### 🚀 Phase 3: 求解、防御与交付 (Prompt_IP_Final.txt)

* **[动作]**: 发送给 AI。此步骤包含核心可视化协议。

```markdown
【系统指令：Phase 3 - 求解、防御与交付】

请编写完整的执行代码块，包含以下步骤：

1.  **求解与妥协 (Solve)**:
    - `solver.solve(time_limit=300, gap_rel=0.05)` (允许 5% Gap 以换取速度)。
    - 增加逻辑判断：若状态为 `Infeasible`，提示我查看诊断报告。

2.  **深度防御 (Deep Analysis)**:
    - **代价分析**: `solver.analyze_relaxation_gap()`。
    - **稳定性测试**: `solver.analyze_binary_stability(perturb_range=0.1)` (仅在有 Binary 变量时运行)。

3.  **场景化交付 (Delivery)**:
    - `solver.export_results()`。
    - **可视化分流 (根据 Phase 1 的场景)**:
        - **CASE A (路径问题)**: 
            - 执行 `solver.visualize_routing(...)`。
            - **务必执行** `solver.export_routing_gephi(matrix_name='你的变量名')`。
        - **CASE B (排班问题)**: 执行 `solver.visualize_schedule(...)`。
        - **CASE C (指派问题)**: 执行 `solver.visualize_matrix(...)`。

4.  **附录操作提示 (Gephi Protocol)**:
    - 如果生成了 `Routing_Flow_Gephi.csv`，请在代码块下方直接输出以下提示给我（作为 Markdown 文本）：
      > " **Gephi 操作指南**: 请打开 Gephi -> Import Spreadsheet。使用 **Force Atlas 2** 布局，并按 **Modularity Class** 染色，即可得到分颜色的最优路由网络图。"
```

---

### 🌟 附录：Gephi 附录协议 (本地操作指南)

**Gephi Routing Protocol**
> 当 Phase 3 生成 `Routing_Flow_Gephi.csv` 后，请在本地 Gephi 软件中执行：

1.  **Import**: 导入 CSV，确保 `Weight` 列被识别为 Edge Weight。
2.  **Layout**: 使用 **Force Atlas 2**，勾选 `Prevent Overlap`。
3.  **Color**: 运行右侧 Statistics -> Modularity，然后在 Appearance 面板按 **Modularity Class** 对边染色（不同颜色的回路）。
4.  **Export**: 导出 SVG 插入论文。